## Making an Test UI using Gradio 🖥️

In [23]:
from ultralytics import YOLO
import numpy as np
import matplotlib.pyplot as plt
import cv2 
import gradio as gr
import pickle


#### following the function to detect the damage from the cars and return the classe of the detected damage

In [24]:
# function which is returning the number of object detected
def number_object_detected(image):
    
    custom_model = YOLO('D:/COMSATS/FYP/Coding Part/ModelMaking/Computer_Vision_Part/3rd Try/runs/detect/train9/weights/best.pt') # custome yolo model path
    results = custom_model(image,verbose= False)

    dic = results[0].names
    classes = results[0].boxes.cls.cpu().numpy()
    probability = results[0].boxes.conf

    class_count = {}
    unique_elements, counts = np.unique(classes, return_counts=True)
    for e , count in zip(unique_elements,counts):
        a = dic[e]
        class_count[a] = count
    """
    for r in results:
        im_array = r.plot(pil = True)  # plot a BGR numpy array of predictions
        array = im_array[..., ::-1]  # Convert BGR to RGB PIL image
        plt.axis("off")
        plt.imshow(array)
        plt.show()
    """


    return class_count

In [25]:
number_object_detected('C:/Users/M_ros/Downloads/Testingimages/s8.jpeg')

{'scratch': 6}

#### following is the code to detect car in the image and crop the image according to it, this will help us to achieve better results from the condition detecting model

In [26]:
def car_detection_and_Cropping(image_path):
    simple_yolo = YOLO('yolov8m.pt')
    r = simple_yolo(image_path,verbose = False)
    """
    for i in r:
        im_array = i.plot(pil=True)  # plot a BGR numpy array of predictions
        array = im_array[..., ::-1]  # Convert BGR to RGB PIL image
        plt.axis("off")
        plt.imshow(array)
        plt.show()
        """

    names = r[0].names
    boxes = r[0].boxes.xyxy.cpu().numpy().astype(int)
    classes = set(r[0].boxes.cls.cpu().numpy())
    classes2 =  [names[i] for i in classes]

    # checking if the detected object is the car or not 
    # if it is car then crop if not then pass the image as it is
    if boxes.size != 0 and 'car' in classes2:
      
      area = []
      for x1, y1, x2, y2 in boxes:
          area.append((x2 - x1) * (y2 - y1))
      max_index, max_a = max(enumerate(area), key=lambda x: x[1])
      

      # Load the image using OpenCV
      image = cv2.imread(image_path)
      
      
      # Crop the image
      crop_image = image[boxes[max_index][1]:boxes[max_index][3], boxes[max_index][0]:boxes[max_index][2]]

      # passing the crop image to the detection model

      class_c = number_object_detected(crop_image)
    else:
      class_c = number_object_detected(image_path)
    return class_c


In [27]:
severity_points = {
    'scratch': 1,
    'dent': 2,
    'rust': 2,
    'paint-damage': 2,
    'crack':2
}

def calculate_condition_score(detections):
    total_score = 0
    for detection, count in detections.items():
        if detection in severity_points:
            total_score += severity_points[detection] * count
    return total_score

def normalize_score(score, max_score):
    return (score / max_score) * 10



## Driving Function💡

In [28]:
## this function will take the image url and call all the related functions
def estimate_condition(detections):
    print("Detedtion list",detections)
    max_possible_score = sum(severity_points.values())  # Assuming all types of damage detected
    score = calculate_condition_score(detections)
    normalized_score = normalize_score(score, max_possible_score)
    
    # Assign condition rating
    if normalized_score <= 10:  # If score is low, condition is Excellent
        return "Excellent"
    elif normalized_score <= 16:  # If score is moderately low, condition is Good
        return "Good"
    elif normalized_score <= 20:  # If score is moderate, condition is Fair 
        return "Fair"
    elif normalized_score <= 258:  # If score is moderately high, condition is Poor
        return "Poor"
    else:  # If score is high, condition is Very Poor
        return "Very Poor"


In [29]:
## loading the model
with open('Price_prediction_decision_tree.pkl', 'rb') as file:
    loaded_pipe_lr = pickle.load(file)

In [30]:
def process_data(files,car_brand, car_name, model_year, mileage, city_registered, color, engine_c, trans, fuel_type, Cate):
    print(car_brand, car_name, model_year, mileage, city_registered, color, engine_c, trans, fuel_type,  Cate)
    # Process uploaded files
    file_names = [f for f in files[0]]
    print(file_names)
    # Process text input
    damage_dic = {}
    
    for i in file_names:
          data = car_detection_and_Cropping(i)
          print(data)
          for key in data.keys():
                      if key in damage_dic:
                        damage_dic[key] += data[key]
                      else:
                          damage_dic[key] = data[key]
    condition = estimate_condition(damage_dic)
    #[[2022,46000 , 'punjba','white',1500,'changan','alsvin','automatic','petrol','excellent','sedan']])
    price = loaded_pipe_lr.predict([[model_year,mileage,city_registered,color,engine_c,car_brand,car_name,trans,fuel_type,condition,Cate]])
    if price[0] >= 100:
          price[0] = price[0]/100
    
    return  price[0]
    


years_list = list(range(2024, 1899, -1))
gr.Interface(fn = process_data,
             inputs=[gr.Gallery(label="Upload Files", type="filepath"),
                  gr.Dropdown(['suzuki','toyota','honda','kia'], label='Brand'),
                      gr.Textbox(lines=1, label="Car Name"),
                      gr.Dropdown(choices=years_list, label='Model Year'),
                      gr.Number(label="Mileage Km"),
                      gr.Textbox(lines=1, label="City Register"),
                      gr.Textbox(lines=1, label="Color"),
                      gr.Number(label="Engine Capacity in CC"),
                      gr.Radio(["automatic", "manual"], label="Transmission Type"),
                      gr.Radio(["hybrid", "petrol",'diesel'], label="Fuel Type"),
                      gr.Radio(["hatchback", "sedan",'suv','croosover'], label="Category")],
                    outputs=gr.Textbox(label="Predicted Price")).launch()

Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


suzuki swift 2018 65000 lahore white 1300 manual petrol hatchback
['C:\\Users\\M_ros\\AppData\\Local\\Temp\\gradio\\281550c7566b8d3f03f705c1e76a3ccad752fbb1\\s9.png', None]
{}
WARNING ⚠️ 'source' is missing. Using 'source=C:\Users\M_ros\anaconda3\envs\fyp2\Lib\site-packages\ultralytics\assets'.
WARNING ⚠️ 'source' is missing. Using 'source=C:\Users\M_ros\anaconda3\envs\fyp2\Lib\site-packages\ultralytics\assets'.
{}
Detedtion list {}


c:\Users\M_ros\anaconda3\envs\fyp2\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


In [31]:
a = 3
b = 3

c = str(a+b) + 'lacs'
c

'6lacs'